<a href="https://colab.research.google.com/github/seobando/UDACITY_GenerativeAI/blob/main/LightweightFineTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
!pip install -q "datasets==2.15.0"

In [27]:
#!pip install -U transformer

In [28]:
!pip install peft

# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: LoRA
* Model: gpt2
* Evaluation approach: accuracy evaluation
* Fine-tuning dataset: imdb

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [29]:
dataset_name= "imdb"
model_name = "gpt2"

In [30]:
# Import the datasets and transformers packages

from datasets import load_dataset

# Load the train and test splits of the imdb dataset
splits = ["train", "test"]
ds = {split: ds for split, ds in zip(splits, load_dataset(dataset_name, split=splits))}

# Thin out the dataset to make it run faster for this example
for split in splits:
    ds[split] = ds[split].shuffle(seed=42).select(range(800))

# Show the dataset
ds

{'train': Dataset({
     features: ['text', 'label'],
     num_rows: 800
 }),
 'test': Dataset({
     features: ['text', 'label'],
     num_rows: 800
 })}

In [31]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

def preprocess_function(examples):
    """Preprocess the emotion dataset by returning tokenized examples."""
    return tokenizer(
        examples["text"],
        padding=True,
        truncation=True,
        return_tensors="pt",
        max_length=128
    )

In [32]:
tokenized_data = {}

for split in splits:
    tokenized_data[split] = ds[split].map(preprocess_function, batched=True)

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2,
    id2label={0: "NEGATIVE", 1: "POSITIVE"},
    label2id={"NEGATIVE": 0, "POSITIVE": 1},
)

model.config.pad_token_id = tokenizer.pad_token_id

for param in model.base_model.parameters():
    param.requires_grad = False

In [ ]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_dir='./logs',
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

In [ ]:
from transformers import Trainer, DataCollatorWithPadding

trainer = Trainer(
    model=model,
    args= training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer, padding="max_length"),
    compute_metrics=compute_metrics
)

In [ ]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

In [ ]:
trainer.save_model("gpt2_model")

## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [ ]:
gpt_model = AutoModelForSequenceClassification.from_pretrained("gpt2_model")

In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    inference_mode=False,
    r=18,
    lora_alpha=32,
    lora_dropout=0.1,
    modules_to_save=["classifier"],
    bias='lora_only',
)

lora_model = get_peft_model(gpt_model, config)

In [ ]:
from peft import LoraConfig, get_peft_model, TaskType

config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=['c_attn', 'c_proj'],
    lora_dropout=0.1,
    bias="none",
    fan_in_fan_out=True,
    task_type=TaskType.SEQ_CLS
)

lora_model = get_peft_model(model, config)

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=50,
    weight_decay=0.01,
    logging_dir='./logs',
    evaluation_strategy="epoch",
    save_strategy="epoch",
    label_names=["labels"],
    load_best_model_at_end=True,
)

In [ ]:
trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer = tokenizer),
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

In [ ]:
lora_model.save_pretrained("gpt-custom-lora")

In [ ]:
tokenizer.save_pretrained("gpt-custom-lora-tokenize")

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [ ]:
from peft import AutoPeftModelForSequenceClassification

lora_model = AutoPeftModelForSequenceClassification.from_pretrained("gpt2_lora")

#tokenizer = AutoTokenizer.from_pretrained("gpt2_lora_tokenizer")

In [ ]:
# Define input text
input_text = [
    "I absolutely loved the cinematography and the performances in this film. It's a masterpiece!",
    "The plot was convoluted, and the acting felt forced. I wouldn't recommend this movie to anyone."
]

# Tokenize input text
inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)

# Perform inference
with torch.no_grad():
    outputs = model(**inputs)

# Convert logits to probabilities
probs = torch.softmax(outputs.logits, dim=1)

# Get predicted labels
predicted_labels = torch.argmax(probs, dim=1)

# Map labels to sentiments
sentiments = ["Negative", "Positive"]

# Print results
for i, text in enumerate(input_text):
    print(f"Text: {text}")
    print(f"Predicted Sentiment: {sentiments[predicted_labels[i].item()]} (Probability: {probs[i][predicted_labels[i]].item()})")
    print()
